In [1]:
#General imports
import pandas as pd
import numpy as np
import math
import os

In [2]:
#Custom files imports
import config

In [3]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split

In [4]:
dir_list = next(os.walk(config.corpus_dir))[1]

In [7]:
df_source = pd.DataFrame()
for d in dir_list:
    csv_path = config.features_data_dir + d + "\scores.csv"
    df = pd.read_csv(csv_path)
    df['language'] = d
    df_source = df_source.append(df)
    

In [8]:
df_source

,Unnamed: 0,Unnamed: 0.1,source,reference,translation,z-score,avg-score,annotators,similarity,language
0,0,0.0,Uchopíte pak zbraň mezi své předloktí a rameno...,you will then grab the weapon between your for...,you then grasp the gun between your forearm an...,-0.675383,60.000000,3,0.961775,cs-en
1,1,1.0,"Ale je-li New York změna, pak je to také znovu...",but if new york is changed then it s also a r...,but if new york is change it is also reinvent...,-0.829403,44.000000,2,0.963354,cs-en
2,2,2.0,"Dlouho a intenzivně jsem během léta přemýšlel,...",i have been thinking over and over again over ...,i have thought long and hard over the course o...,0.803185,96.500000,2,0.848682,cs-en
3,3,3.0,"Najdou si jiný způsob, jak někde podvádět.",they find another way to cheat somewhere,they will find another way how to defraud others,0.563149,90.500000,2,0.860908,cs-en
4,4,4.0,Zpráva o výměně v čele prezidentovy administra...,the report on the replacement of the president...,the news of the replacement at the top of the ...,0.021549,74.666667,3,0.822409,cs-en
...,...,...,...,...,...,...,...,...,...,...
95,95,95.0,作为运动员，张彬彬和家人聚少离多，只有节假日才能回家，平时多靠电话联系。,as an athlete zhang binbin and his family gat...,as an athlete zhang binbin and her family are...,0.060832,78.500000,2,0.858316,zh-en
96,96,96.0,南湖公园水域管理科科长郑建军介绍，南湖水底最深处超过10米，不了解情况下水者，很可能发生危险。,the south lake park water management section ...,according to zheng jianjun the management sec...,0.273378,78.500000,2,0.779622,zh-en
97,97,97.0,工厂的最大股东是一家美铝公司和氧化铝公司的合资公司，而中国的中信集团和日本的丸红株式会社也持...,the biggest shareholder of the factory is a jo...,the plant is majority owned by a joint venture...,-0.622387,58.000000,2,0.923329,zh-en
98,98,98.0,Banff 的一位居民发现有鱼惊现自家花园，随后发出了求助。,a resident of banff found fish in his garden a...,a banff man has appealed for help in identifyi...,-0.537770,65.500000,2,0.845909,zh-en


In [9]:
X = df_source['similarity']
y = df_source['avg-score']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [10]:
def build_and_compile_model(norm):
    model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
      ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [12]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(X_train))
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

[<tf.Tensor 'Placeholder:0' shape=(None, 1) dtype=float32>]


In [13]:
dnn_model = build_and_compile_model(normalizer)

[<tf.Tensor 'Placeholder:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'Placeholder:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'Placeholder:0' shape=(None, 64) dtype=float32>]


In [14]:
dnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 1)                 3         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                128       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 4,356
Trainable params: 4,353
Non-trainable params: 3
_________________________________________________________________


In [15]:
%%time
history = dnn_model.fit(
   X_train, y_train,
    validation_split=0.2,
    verbose=0, epochs=100)

[<tf.Tensor 'sequential_1/normalization_1/truediv:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'sequential_1/dense_1/Relu:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'sequential_1/dense_2/Relu:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'sequential_1/normalization_1/truediv:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'sequential_1/dense_1/Relu:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'sequential_1/dense_2/Relu:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'sequential_1/normalization_1/truediv:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'sequential_1/dense_1/Relu:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'sequential_1/dense_2/Relu:0' shape=(None, 64) dtype=float32>]
Wall time: 8.89 s


In [19]:
dnn_model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 3ms/step - loss: 19.0496


19.049598693847656

In [20]:
dnn_model.save("dnn_model")

[<tf.Tensor 'sequential_1/normalization_1/truediv:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'sequential_1/dense_1/Relu:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'sequential_1/dense_2/Relu:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'normalization_1/truediv:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'dense_1/StatefulPartitionedCall:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'dense_2/StatefulPartitionedCall:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'normalization_1/truediv:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'dense_1/StatefulPartitionedCall:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'dense_2/StatefulPartitionedCall:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'normalization_1/truediv:0' shape=(None, 1) dtype=float32>]
[<tf.Tensor 'dense_1/StatefulPartitionedCall:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'dense_2/StatefulPartitionedCall:0' shape=(None, 64) dtype=float32>]
[<tf.Tensor 'normalization_1/truediv:0' shape=(None, 1) dtype=f